# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4880, uptime 0:00:16
mini_beamline                    RUNNING   pid 4881, uptime 0:00:16
random_walk                      RUNNING   pid 4882, uptime 0:00:16
random_walk_horiz                RUNNING   pid 4883, uptime 0:00:16
random_walk_vert                 RUNNING   pid 4884, uptime 0:00:16
simple                           RUNNING   pid 4885, uptime 0:00:16
thermo_sim                       RUNNING   pid 4886, uptime 0:00:16
trigger_with_pc                  RUNNING   pid 4887, uptime 0:00:16


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-17 19:53:32
Persistent Unique Scan ID: '9b626fab-d410-4909-ac89-75ff314a1314'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 19:53:32.7 |     -1.000 |      96183 |
|         2 | 19:53:32.7 |     -0.500 |      98666 |
|         3 | 19:53:32.7 |      0.000 |      99123 |
|         4 | 19:53:32.8 |      0.500 |      98119 |
|         5 | 19:53:32.8 |      1.000 |      97027 |


+-----------+------------+------------+------------+
generator scan ['9b626fab'] (scan num: 1)





('9b626fab-d410-4909-ac89-75ff314a1314',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/9b626fab-d410-4909-ac89-75ff314a1314-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-17 19:53:33
Persistent Unique Scan ID: '8117dc6e-54de-4849-8991-8840eae82dc7'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 19:53:33.2 |          -1 | 1100840904 |


|         2 | 19:53:33.4 |          -0 | 1109756655 |


|         3 | 19:53:33.5 |           0 | 1120160627 |
|         4 | 19:53:33.6 |           0 | 1124252024 |


|         5 | 19:53:33.7 |           1 | 1127470709 |


+-----------+------------+-------------+------------+
generator scan ['8117dc6e'] (scan num: 2)





('8117dc6e-54de-4849-8991-8840eae82dc7',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-17 19:53:34
Persistent Unique Scan ID: '2e471b84-a278-4656-b4fd-00704ddda9d0'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 19:53:34.2 | 1129841618 |


|         2 | 19:53:35.2 | 1080109823 |


|         3 | 19:53:36.2 | 1055357857 |


...

|        28 | 19:54:01.2 | 1106255056 |


|        29 | 19:54:02.2 | 1129192762 |


|        30 | 19:54:03.2 | 1078203202 |


+-----------+------------+------------+
generator count ['2e471b84'] (scan num: 3)





('2e471b84-a278-4656-b4fd-00704ddda9d0',)